In [2]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.9 MB/s eta 0:00:00


In [3]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for
import math #The Python math module
from scipy import stats #The SciPy stats module

In [4]:
stocks = pd.read_csv('NIFTY_50_stocks.csv', skiprows=1)
print(stocks.head())

   S.NO.   SYMBOL \n
0      0  APOLLOHOSP
1      1    HINDALCO
2      2   ICICIBANK
3      3        INFY
4      4  BAJAJFINSV


In [5]:
# Read the CSV file while skipping the first row
data = pd.read_csv('Nifty_50_Data.csv', skiprows=1)

# Add the 'Number of Shares to Buy' column with initial value 'N/A'
data['Number of Shares to Buy'] = 'N/A'

In [6]:
# Remove '\n' from column names
data.columns = data.columns.str.replace('\n', '')
data.head()

,SYMBOL,Price,VOLUME (shares),52W H,52W L,30 D %CHNG,365 D % CHNG,Number of Shares to Buy
0,APOLLOHOSP,"6,505.10","5,92,624","6,874.45","4,187.30",4.28,50.11,N/A
1,HINDALCO,588.60,"78,29,711",620.50,397.80,7.47,42.33,N/A
2,ICICIBANK,"1,107.70","1,22,98,463","1,115.25",864.40,-0.07,24.25,N/A
3,INFY,"1,497.25","94,23,230","1,733.00","1,185.30",-8.65,3.85,N/A
4,BAJAJFINSV,"1,708.00","12,84,145","1,741.00","1,295.65",6.72,29.54,N/A


In [7]:
data.columns = data.columns.str.strip()
print(data.columns)

Index(['SYMBOL', 'Price', 'VOLUME (shares)', '52W H', '52W L', '30 D   %CHNG',
       '365 D % CHNG', 'Number of Shares to Buy'],
      dtype='object')


In [8]:
# Create DataFrame
df = pd.DataFrame(data)

# Convert columns to numeric
df['Price'] = pd.to_numeric(df['Price'].str.replace(',', ''))
df['30 D   %CHNG'] = pd.to_numeric(df['30 D   %CHNG'])
df['365 D % CHNG'] = pd.to_numeric(df['365 D % CHNG'])

# Calculate price returns
df['1-month price returns'] = df['Price'] * (1 + df['30 D   %CHNG'] / 100)
df['3-month price returns'] = df['Price'] * (1 + df['30 D   %CHNG'] / 100) ** 3
df['6-month price returns'] = df['Price'] * (1 + df['30 D   %CHNG'] / 100) ** 6
df['1-year price returns'] = df['Price'] * (1 + df['365 D % CHNG'] / 100)

In [9]:
df.head()

,SYMBOL,Price,VOLUME (shares),52W H,52W L,30 D %CHNG,365 D % CHNG,Number of Shares to Buy,1-month price returns,3-month price returns,6-month price returns,1-year price returns
0,APOLLOHOSP,6505.10,"5,92,624","6,874.45","4,187.30",4.28,50.11,N/A,6783.518280,7376.613765,8364.887648,9764.805610
1,HINDALCO,588.60,"78,29,711",620.50,397.80,7.47,42.33,N/A,632.568420,730.603931,906.867318,837.754380
2,ICICIBANK,1107.70,"1,22,98,463","1,115.25",864.40,-0.07,24.25,N/A,1106.924610,1105.375458,1103.055794,1376.317250
3,INFY,1497.25,"94,23,230","1,733.00","1,185.30",-8.65,3.85,N/A,1367.737875,1141.352979,870.052846,1554.894125
4,BAJAJFINSV,1708.00,"12,84,145","1,741.00","1,295.65",6.72,29.54,N/A,1822.777600,2075.990281,2523.264431,2212.543200


In [10]:
# Calculate momentum percentiles
time_periods = ['1-month', '3-month', '6-month', '1-year']
for row in df.index:
    for time_period in time_periods:
        percentile = stats.percentileofscore(df[f'{time_period} price returns'], df.loc[row, f'{time_period} price returns']) / 100
        df.loc[row, f'{time_period} Return Percentile'] = percentile

In [11]:
from statistics import mean
# Calculate HQM Score
for row in df.index:
    momentum_percentiles = [df.loc[row, f'{time_period} Return Percentile'] for time_period in time_periods]
    df.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [12]:
# Select top 50 stocks based on HQM Score
hqm_dataframe = df.sort_values(by='HQM Score', ascending=False).head(5).reset_index(drop=True)

In [13]:
hqm_dataframe.head()

,SYMBOL,Price,VOLUME (shares),52W H,52W L,30 D %CHNG,365 D % CHNG,Number of Shares to Buy,1-month price returns,3-month price returns,6-month price returns,1-year price returns,1-month Return Percentile,3-month Return Percentile,6-month Return Percentile,1-year Return Percentile,HQM Score
0,MARUTI,12868.25,"3,14,466","12,985.70","8,403.05",11.77,51.06,N/A,14382.843025,17967.813969,25088.286193,19438.77845,1.00,1.00,1.00,0.98,0.995
1,ULTRACEMCO,9770.00,"2,75,924","10,526.00","7,300.00",1.97,28.07,N/A,9962.469000,10358.856613,10983.204742,12512.43900,0.98,0.98,0.96,0.96,0.970
2,BAJAJ-AUTO,8990.00,"2,37,944","9,357.95","4,030.10",1.71,123.88,N/A,9143.729000,9459.118250,9952.716136,20126.81200,0.96,0.94,0.94,1.00,0.960
3,BAJFINANCE,7189.00,"7,10,281","8,192.00","5,786.00",11.80,21.01,N/A,8037.302000,10046.016665,14038.454699,8699.40890,0.94,0.96,0.98,0.92,0.950
4,APOLLOHOSP,6505.10,"5,92,624","6,874.45","4,187.30",4.28,50.11,N/A,6783.518280,7376.613765,8364.887648,9764.80561,0.92,0.92,0.92,0.94,0.925


In [14]:
# Calculate number of shares to buy
portfolio_size = float(input("Enter the value of your portfolio: "))

Enter the value of your portfolio: 100000


In [15]:
position_size = portfolio_size / len(hqm_dataframe.index)
hqm_dataframe['Number of Shares to Buy'] = (position_size / hqm_dataframe['Price']).apply(math.floor)

In [16]:
hqm_dataframe

,SYMBOL,Price,VOLUME (shares),52W H,52W L,30 D %CHNG,365 D % CHNG,Number of Shares to Buy,1-month price returns,3-month price returns,6-month price returns,1-year price returns,1-month Return Percentile,3-month Return Percentile,6-month Return Percentile,1-year Return Percentile,HQM Score
0,MARUTI,12868.25,"3,14,466","12,985.70","8,403.05",11.77,51.06,1,14382.843025,17967.813969,25088.286193,19438.77845,1.00,1.00,1.00,0.98,0.995
1,ULTRACEMCO,9770.00,"2,75,924","10,526.00","7,300.00",1.97,28.07,2,9962.469000,10358.856613,10983.204742,12512.43900,0.98,0.98,0.96,0.96,0.970
2,BAJAJ-AUTO,8990.00,"2,37,944","9,357.95","4,030.10",1.71,123.88,2,9143.729000,9459.118250,9952.716136,20126.81200,0.96,0.94,0.94,1.00,0.960
3,BAJFINANCE,7189.00,"7,10,281","8,192.00","5,786.00",11.80,21.01,2,8037.302000,10046.016665,14038.454699,8699.40890,0.94,0.96,0.98,0.92,0.950
4,APOLLOHOSP,6505.10,"5,92,624","6,874.45","4,187.30",4.28,50.11,3,6783.518280,7376.613765,8364.887648,9764.80561,0.92,0.92,0.92,0.94,0.925


In [17]:
hqm_dataframe.columns

Index(['SYMBOL', 'Price', 'VOLUME (shares)', '52W H', '52W L', '30 D   %CHNG',
       '365 D % CHNG', 'Number of Shares to Buy', '1-month price returns',
       '3-month price returns', '6-month price returns',
       '1-year price returns', '1-month Return Percentile',
       '3-month Return Percentile', '6-month Return Percentile',
       '1-year Return Percentile', 'HQM Score'],
      dtype='object')

In [18]:
# Define the file name
file_name = "momentum_strategy.xlsx"

# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Write the DataFrame to the Excel file
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

# Get the xlsxwriter workbook and worksheet objects
workbook  = writer.book
worksheet = writer.sheets['Momentum Strategy']

In [19]:
## Define formats for the columns
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = workbook.add_format({'font_color': font_color, 'bg_color': background_color, 'border': 1})
rupee_template = workbook.add_format({'num_format': '\u20B9#,##0.00', 'font_color': font_color, 'bg_color': background_color, 'border': 1})
integer_template = workbook.add_format({'num_format': '0', 'font_color': font_color, 'bg_color': background_color, 'border': 1})
percent_template = workbook.add_format({'num_format': '0.0%', 'font_color': font_color, 'bg_color': background_color, 'border': 1})

In [20]:
workbook  = writer.book
# Define column formats
column_formats = {
    'A': ['SYMBOL', string_template],
    'B': ['Price', rupee_template],
    'C': ['VOLUME (shares)', integer_template],
    'D': ['52W H', rupee_template],
    'E': ['52W L', rupee_template],
    'F': ['30 D   %CHNG', percent_template],
    'G': ['365 D % CHNG', percent_template],
    'H': ['Number of Shares to Buy', integer_template],
    'I': ['1-month price returns', percent_template],
    'J': ['3-month price returns', percent_template],
}

# Set column formats
for column, (header, format_) in column_formats.items():
    worksheet.set_column(f'{column}:{column}', 20, format_)
    worksheet.write(f'{column}1', header, string_template)

In [21]:
# Save the Excel file
workbook.close()
from google.colab import files

files.download('momentum_strategy.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>